In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyrender

from face_reconstruction.graphics import SimpleImageRenderer, draw_pixels_to_image
from face_reconstruction.landmarks import load_bfm_landmarks
from face_reconstruction.model import BaselFaceModel

# 1. Load Basel Face model

In [ ]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")
bfm_landmarks = load_bfm_landmarks("model2019_face12_landmarks_v2")
bfm_landmark_indices = list(bfm_landmarks.values())

In [ ]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

In [ ]:
shape_coefficients = [0 for _ in range(n_shape_coefficients)]
expression_coefficients = [0 for _ in range(n_expression_coefficients)]
color_coefficients = [0 for _ in range(n_color_coefficients)]

face_mesh = bfm.draw_sample(shape_coefficients, expression_coefficients, color_coefficients)
face_trimesh = bfm.convert_to_trimesh(face_mesh)

In [ ]:
# This is how we can reconstruct the draw_sample method in case we need differentiable rendering
reconstructed = bfm.shape_model.get_mean() + bfm.expression_model.get_mean()
reconstructed = reconstructed.reshape((-1, 3))

In [ ]:
img_width = 480
img_height = 640

# 2. Render the face model

In [ ]:
perspective_camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
projection_matrix = perspective_camera.get_projection_matrix(width=img_width, height=img_height)
projection_matrix

In [ ]:
initial_camera_pose = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, -300], [0, 0, 0, 1]]) # position camera just in front of face

In [ ]:
renderer = SimpleImageRenderer(projection_matrix, img_width, img_height)

In [ ]:
test_pixels = renderer.project_points(initial_camera_pose, face_mesh.vertices)

test_img = np.zeros((img_height, img_width))
draw_pixels_to_image(test_img, test_pixels)
plt.figure(figsize=(12, 8))
plt.imshow(test_img)
plt.show()